# Applied Data Science Capstone

## Week 3 Assignment : Part 3 : Segmenting and Clustering Toronto Neighborhoods

### Author : Bill Gourley
### Date     : 30/11/2018

#### Description :  This notebook combines Parts 1 and 2 of the assignment and adds Analysis of the Toronto Neighborhoods and venues.

##### Note that the Foursquare API credentials have been masked for security. If you wish to replicate the results, then insert your own credentials.

In [574]:
#CLIENT_ID = '##########' # your Foursquare ID
#CLIENT_SECRET = '############' # your Foursquare Secret
#VERSION = '20180605' # Foursquare API version

In [575]:
#import required libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import csv
from geopy.geocoders import Nominatim
import folium
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

pd.options.display.max_columns = None
pd.options.display.max_rows = None

##### Download Wikipedia Page

In [576]:
result = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

##### Parse the result content into a BeautifulSoup object

In [577]:
soup = BeautifulSoup(result.content,'lxml')

##### Extract the table from the page (as there is only 1 table, we do not need to use find_all)

In [578]:
table = soup.find('table',{'class':'wikitable sortable'})

##### Extract the rows from the table and append to a list (each extracted row is also a list)

In [579]:
table_rows = table.find_all('tr')
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)

##### The first list entry is empty as we have not extracted the headers (as there is only 3, they will be added manually to the dataframe

In [580]:
#input row data
del(l[0])

##### Create a Pandas DataFrame from the extracted row list, and add the column names

In [581]:
#create dataframe
columns=['Postcode','Borough','Neighborhood']
df = pd.DataFrame(l,columns=columns)
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


##### Clean the Neighborhood column by removing the trailing newline string

In [582]:
#remove \n from Neighborhood column
df.Neighborhood = df.Neighborhood.str.replace('\n','')

##### There are 289 rows and 3 columns in the dataframe before further cleaning

In [583]:
#shape of dataframe before dropping rows
df.shape

(289, 3)

##### Clean dataframe by dropping any row which has a 'Not assigned' value for Borough

In [584]:
#drop all rows where Borough = 'Not assigned'
df = df.drop(df[df.Borough == 'Not assigned'].index)

##### This leaves 212 rows and 3 columns in the dataframe

In [585]:
#shape of dataframe after dropping rows
df.shape

(212, 3)

##### Join Neighborhood values with a comma where Postcode and Borough are identical and remove duplicate rows

In [586]:
#join Neighborhood values with a comma where Postcode and Borough are identical
df = df.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

##### Assign Neighborhood value to Borough where Neighborhood is 'Not assigned' and check there are no 'Not assigned' Neighborhoods remaining

In [587]:
#Assign Neighborhood value to Borough where Neighborhood is 'Not assigned'
df.Neighborhood = np.where(df.Neighborhood == 'Not assigned',df.Borough,df.Neighborhood)

#check there are no 'Not assigned' Neighborhoods remaining
print(df.loc[df.Neighborhood == 'Not assigned'].count())

Postcode        0
Borough         0
Neighborhood    0
dtype: int64


##### First 5 rows of the cleaned dataframe

In [588]:
df.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


##### The cleaned dataframe contains 103 rows and 3 columns

In [589]:
#shape of the cleaned dataframe
df.shape

(103, 3)

##### Download Geospatial_data csv file and extract contents into a list

In [590]:
# Download Geospatial_data csv file and extract contents into a list
csv_url = 'http://cocl.us/Geospatial_data'
with requests.Session() as s:
    download = s.get(csv_url)

    decoded_content = download.content.decode('utf-8')

    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    latlon_list = list(cr)

##### Remove first list element containing header information and create a dataframe with the contents of the list

In [591]:
# Remove first list element containing header information and create a dataframe with the contents of the list
del latlon_list[0]
columns = ['Postcode','Latitude','Longitude']
df_latlon = pd.DataFrame(latlon_list,columns = columns)

##### Check shape of the new datframe is the same as the cleaned dataframe

In [592]:
# Check shape of the new datframe is the same as the cleaned dataframe
df_latlon.shape

(103, 3)

##### Merge the latitude and longitude dataframe with the original cleaned dataframe 

In [593]:
# Merge the latitude and longitude dataframe with the original cleaned dataframe 
df_toronto = pd.merge(df, df_latlon, on='Postcode')

##### First 5 rows of merged dataframe

In [594]:
# First 5 rows of merged dataframe
df_toronto.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8066863,-79.1943534
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845351,-79.1604971
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7635726,-79.1887115
3,M1G,Scarborough,Woburn,43.7709921,-79.2169174
4,M1H,Scarborough,Cedarbrae,43.773136,-79.2394761


### Toronto Venue Analysis

##### First ensure that Latitude and Longitude values are set to type float

In [595]:
#Change type of Latitude and Longitude to float
df_toronto.Latitude = df_toronto.Latitude.astype(float)
df_toronto.Longitude = df_toronto.Longitude.astype(float)
df_toronto.dtypes

Postcode         object
Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

##### Count of Toronto Boroughs and Neighborhoods 

In [596]:
print('The df_toronto dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_toronto.Borough.unique()),
        df_toronto.shape[0]
    )
)

The df_toronto dataframe has 11 boroughs and 103 neighborhoods.


##### Get the coordinates of Toronto for use in creating the map

In [597]:
address = 'Toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


C:\Users\bgour\Anaconda3\lib\site-packages\geopy\geocoders\osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


##### Create Map of Toronto

In [598]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto.Latitude, df_toronto.Longitude, df_toronto.Borough, 
                                           df_toronto.Neighborhood):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        ).add_to(map_toronto)  
    
map_toronto

##### Function definition for getNearbyVenues()

In [599]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

##### Get number of unique Venue Categories

In [600]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 273 unique categories.


##### Perform One Hot Encoding to prepare for Cluster Analysis

In [601]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

##### shape confirms that thete are 273 unique categories

In [602]:
toronto_onehot.shape

(2253, 273)

##### Group by Neighbourhood and report means

In [603]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

##### shape indicates that there are 100 grouped Neighborhoods (3 Neighborhoods less than the number in the original dataset)

In [604]:
toronto_grouped.shape

(100, 273)

##### Calculate the top 5 venues for each Neighborhood

In [605]:
num_top_venues = 5

for hood in toronto_grouped.Neighborhood:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped.Neighborhood == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.05
2           Steakhouse  0.04
3      Thai Restaurant  0.04
4  American Restaurant  0.04


----Agincourt----
                    venue  freq
0                  Lounge  0.25
1  Furniture / Home Store  0.25
2            Skating Rink  0.25
3          Breakfast Spot  0.25
4             Yoga Studio  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
              venue  freq
0              Park  0.33
1        Playground  0.33
2  Sculpture Garden  0.33
3       Yoga Studio  0.00
4     Metro Station  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0          Pizza Place  0.15
1        Grocery Store  0.15
2       Discount Store  0.08
3          Video Store  0.08
4  Fried Chicken Joint  0.08


----Alderwood, Long Branch----
          venue  freq
0   Pizza


----East Toronto----
                venue  freq
0                Park  0.50
1         Coffee Shop  0.25
2   Convenience Store  0.25
3         Yoga Studio  0.00
4  Mexican Restaurant  0.00


----Emery, Humberlea----
                             venue  freq
0                   Baseball Field   1.0
1                      Yoga Studio   0.0
2                    Metro Station   0.0
3              Monument / Landmark   0.0
4  Molecular Gastronomy Restaurant   0.0


----Fairview, Henry Farm, Oriole----
                  venue  freq
0        Clothing Store  0.12
1  Fast Food Restaurant  0.09
2           Coffee Shop  0.06
3            Restaurant  0.05
4            Food Court  0.03


----First Canadian Place, Underground city----
         venue  freq
0  Coffee Shop  0.11
1         Café  0.08
2        Hotel  0.06
3   Restaurant  0.05
4   Steakhouse  0.04


----Flemingdon Park, Don Mills South----
              venue  freq
0       Coffee Shop  0.10
1               Gym  0.10
2        Beer Store  0

----Runnymede, Swansea----
                venue  freq
0         Coffee Shop  0.11
1         Pizza Place  0.08
2                Café  0.08
3    Sushi Restaurant  0.05
4  Italian Restaurant  0.05


----Ryerson, Garden District----
                       venue  freq
0             Clothing Store  0.08
1                Coffee Shop  0.08
2             Cosmetics Shop  0.04
3                       Café  0.04
4  Middle Eastern Restaurant  0.03


----Scarborough Village----
                 venue  freq
0           Playground   0.5
1    Convenience Store   0.5
2          Yoga Studio   0.0
3        Metro Station   0.0
4  Monument / Landmark   0.0


----Silver Hills, York Mills----
                 venue  freq
0            Cafeteria   0.5
1    Martial Arts Dojo   0.5
2   Mexican Restaurant   0.0
3                Motel   0.0
4  Monument / Landmark   0.0


----St. James Town----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.06
2   Restaurant  0.06
3        Hotel  0.05
4    Gastropub  0

##### Function definition for getting the most common venues

In [606]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

##### Create a dataframe containing the top 10 venues for each neighborhood

In [607]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,American Restaurant,Thai Restaurant,Steakhouse,Hotel,Restaurant,Gym,Bar,Clothing Store
1,Agincourt,Furniture / Home Store,Skating Rink,Breakfast Spot,Lounge,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Falafel Restaurant,Dim Sum Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Sculpture Garden,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Beer Store,Fried Chicken Joint,Japanese Restaurant,Fast Food Restaurant,Discount Store,Pharmacy,Sandwich Place,Coffee Shop
4,"Alderwood, Long Branch",Pizza Place,Athletics & Sports,Pharmacy,Sandwich Place,Bank,Pub,Pool,Skating Rink,Gym,Coffee Shop
5,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Shopping Mall,Pizza Place,Deli / Bodega,Middle Eastern Restaurant,Sushi Restaurant,Bank,Fried Chicken Joint,Frozen Yogurt Shop,Diner
6,Bayview Village,Bank,Café,Japanese Restaurant,Chinese Restaurant,Diner,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store
7,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sushi Restaurant,Fast Food Restaurant,Breakfast Spot,Juice Bar,Indian Restaurant,Pub,Café,Restaurant
8,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Pub,Steakhouse,Farmers Market,Seafood Restaurant,Café,Cheese Shop,Bakery
9,"Birch Cliff, Cliffside West",General Entertainment,College Stadium,Skating Rink,Café,Women's Store,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop


##### Perform Clustering to create 5 clusters

In [608]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

##### List and drop the 3 Neighborhood rows from the orginal dataset before merging with the grouped dataframe

In [609]:
diffs = df_toronto.Neighborhood.isin(toronto_grouped.Neighborhood)
to_drop = []
for count in range(0,len(diffs)):
    if(diffs[count] != True):
        print(count,df_toronto.iloc[count]['Neighborhood'])
        to_drop.append(count)

df_toronto = df_toronto.drop(to_drop).reset_index()

16 Upper Rouge
21 Newtonbrook, Willowdale
93 Islington Avenue


##### Merge the dataframes

In [610]:
toronto_merged = df_toronto

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,index,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0,Fast Food Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Wings Joint
1,1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0,Bar,Construction & Landscaping,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Women's Store
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,Rental Car Location,Electronics Store,Pizza Place,Mexican Restaurant,Breakfast Spot,Medical Center,Women's Store,Donut Shop,Discount Store,Dog Run
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Korean Restaurant,Women's Store,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Electronics Store
4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Athletics & Sports,Fried Chicken Joint,Hakka Restaurant,Bakery,Thai Restaurant,Caribbean Restaurant,Bank,Donut Shop,Discount Store,Dog Run


##### Create the map of Toronto with the venue clusters shown

In [611]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged.Latitude, toronto_merged.Longitude, toronto_merged.Neighborhood, 
                                  toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##### get the number of Neighborhoods in each cluster. From the summary shown, we can see that 98 out of the 100 Neighborhoods belong to clusters 0, 1 and 3 with 86 belonging to cluster 0.

In [612]:
toronto_merged['Cluster Labels'].value_counts()

0    86
1     9
3     3
4     1
2     1
Name: Cluster Labels, dtype: int64

##### A brief analysis of the top 3 clusters (shown below) indicates that the most prominent venue types across the clusters are Coffee shops, food outlets and restaurants. From the map above, Cluster 0 which contains the most venues has a concentration of venues in Downtown which I would suspect, account for the majority of the coffee shops, food outlets and restaurants. 

In [613]:
df_cluster_0 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[3] + list(range(7, toronto_merged.shape[1]))]]
df_cluster_0

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge, Malvern",Fast Food Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Wings Joint
1,"Highland Creek, Rouge Hill, Port Union",Bar,Construction & Landscaping,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Women's Store
3,Woburn,Coffee Shop,Korean Restaurant,Women's Store,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Electronics Store
4,Cedarbrae,Athletics & Sports,Fried Chicken Joint,Hakka Restaurant,Bakery,Thai Restaurant,Caribbean Restaurant,Bank,Donut Shop,Discount Store,Dog Run
5,Scarborough Village,Playground,Convenience Store,Women's Store,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
6,"East Birchmount Park, Ionview, Kennedy Park",Train Station,Department Store,Coffee Shop,Discount Store,Convenience Store,Bus Station,Chinese Restaurant,Drugstore,Diner,Dog Run
7,"Clairlea, Golden Mile, Oakridge",Bus Line,Bakery,Soccer Field,Park,Metro Station,Intersection,Fast Food Restaurant,Cosmetics Shop,Coworking Space,Comfort Food Restaurant
8,"Cliffcrest, Cliffside, Scarborough Village West",Motel,Intersection,American Restaurant,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
9,"Birch Cliff, Cliffside West",General Entertainment,College Stadium,Skating Rink,Café,Women's Store,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
10,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Pet Store,Latin American Restaurant,Vietnamese Restaurant,Furniture / Home Store,Chinese Restaurant,Women's Store,Donut Shop,Diner,Discount Store


In [614]:
columns = []
for count in range(1,11):
    columns.append(df_cluster_0.columns[count])
    
for col in columns:
    counts = df_cluster_0[col].value_counts()
    print(col, ' : ', counts)
    print('')

1st Most Common Venue  :  Coffee Shop                  20
Park                         11
Fast Food Restaurant          4
Grocery Store                 4
Furniture / Home Store        3
Indian Restaurant             2
Bank                          2
Bus Line                      2
Bar                           2
Gym / Fitness Center          2
Pizza Place                   2
Baseball Field                2
Café                          2
Asian Restaurant              2
Yoga Studio                   1
Playground                    1
River                         1
Italian Restaurant            1
Dessert Shop                  1
Motel                         1
Restaurant                    1
Japanese Restaurant           1
General Entertainment         1
Athletics & Sports            1
Liquor Store                  1
Greek Restaurant              1
Clothing Store                1
Ramen Restaurant              1
Hotel                         1
Garden                        1
Gym           

In [615]:
df_cluster_1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[3] + list(range(7, toronto_merged.shape[1]))]]
df_cluster_1

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Guildwood, Morningside, West Hill",Rental Car Location,Electronics Store,Pizza Place,Mexican Restaurant,Breakfast Spot,Medical Center,Women's Store,Donut Shop,Discount Store,Dog Run
13,"Clarks Corners, Sullivan, Tam O'Shanter",Pizza Place,Rental Car Location,Fried Chicken Joint,Italian Restaurant,Thai Restaurant,Noodle House,Chinese Restaurant,Drugstore,Dumpling Restaurant,Donut Shop
16,Hillcrest Village,Pool,Mediterranean Restaurant,Dog Run,Golf Course,Athletics & Sports,Diner,Discount Store,Doner Restaurant,Donut Shop,Drugstore
40,"The Beaches West, India Bazaar",Park,Pizza Place,Brewery,Sandwich Place,Fast Food Restaurant,Burger Joint,Fish & Chips Shop,Italian Restaurant,Burrito Place,Steakhouse
58,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Café,Hotel,American Restaurant,Restaurant,Italian Restaurant,Gastropub,Deli / Bodega,Gym,Thai Restaurant
74,"Dovercourt Village, Dufferin",Bakery,Discount Store,Pharmacy,Supermarket,Gym / Fitness Center,Music Venue,Brewery,Bar,Bank,Middle Eastern Restaurant
89,"Humber Bay, King's Mill Park, Kingsway Park So...",Baseball Field,Women's Store,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
93,Humber Summit,Pizza Place,Empanada Restaurant,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
99,Northwest,Drugstore,Rental Car Location,Bar,Women's Store,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


In [616]:
columns = []
for count in range(1,11):
    columns.append(df_cluster_1.columns[count])
    
for col in columns:
    counts = df_cluster_1[col].value_counts()
    print(col, ' : ', counts)
    print('')

1st Most Common Venue  :  Pizza Place            2
Pool                   1
Bakery                 1
Coffee Shop            1
Drugstore              1
Rental Car Location    1
Baseball Field         1
Park                   1
Name: 1st Most Common Venue, dtype: int64

2nd Most Common Venue  :  Rental Car Location         2
Electronics Store           1
Women's Store               1
Mediterranean Restaurant    1
Discount Store              1
Café                        1
Pizza Place                 1
Empanada Restaurant         1
Name: 2nd Most Common Venue, dtype: int64

3rd Most Common Venue  :  Brewery                        1
Bar                            1
Fried Chicken Joint            1
Women's Store                  1
Dog Run                        1
Pharmacy                       1
Pizza Place                    1
Eastern European Restaurant    1
Hotel                          1
Name: 3rd Most Common Venue, dtype: int64

4th Most Common Venue  :  Golf Course            1
Ameri

In [617]:
df_cluster_3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[3] + list(range(7, toronto_merged.shape[1]))]]
df_cluster_3

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,Leaside,Sporting Goods Shop,Coffee Shop,Grocery Store,Burger Joint,Clothing Store,Restaurant,Sandwich Place,Bagel Shop,Bank,Fish & Chips Shop
41,Studio District,Café,Coffee Shop,Gastropub,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Convenience Store,Seafood Restaurant,Brewery
55,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Bubble Tea Shop,Bar,Burger Joint,Chinese Restaurant,Ice Cream Shop,Indian Restaurant


In [618]:
columns = []
for count in range(1,11):
    columns.append(df_cluster_3.columns[count])
    
for col in columns:
    counts = df_cluster_3[col].value_counts()
    print(col, ' : ', counts)
    print('')

1st Most Common Venue  :  Café                   1
Coffee Shop            1
Sporting Goods Shop    1
Name: 1st Most Common Venue, dtype: int64

2nd Most Common Venue  :  Coffee Shop    2
Café           1
Name: 2nd Most Common Venue, dtype: int64

3rd Most Common Venue  :  Grocery Store     1
Gastropub         1
Sandwich Place    1
Name: 3rd Most Common Venue, dtype: int64

4th Most Common Venue  :  Italian Restaurant    1
Burger Joint          1
Bakery                1
Name: 4th Most Common Venue, dtype: int64

5th Most Common Venue  :  Clothing Store        1
Italian Restaurant    1
Bubble Tea Shop       1
Name: 5th Most Common Venue, dtype: int64

6th Most Common Venue  :  American Restaurant    1
Bar                    1
Restaurant             1
Name: 6th Most Common Venue, dtype: int64

7th Most Common Venue  :  Yoga Studio       1
Burger Joint      1
Sandwich Place    1
Name: 7th Most Common Venue, dtype: int64

8th Most Common Venue  :  Convenience Store     1
Bagel Shop         